<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Init" data-toc-modified-id="Init-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Init</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Helper functions</a></span></li></ul></li><li><span><a href="#Descriptive-stats" data-toc-modified-id="Descriptive-stats-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Descriptive stats</a></span><ul class="toc-item"><li><span><a href="#Summary-table" data-toc-modified-id="Summary-table-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Summary table</a></span></li><li><span><a href="#How-many-imputed-items?" data-toc-modified-id="How-many-imputed-items?-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>How many imputed items?</a></span></li><li><span><a href="#Time-series-plots" data-toc-modified-id="Time-series-plots-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Time series plots</a></span></li></ul></li><li><span><a href="#TWFE-estimator" data-toc-modified-id="TWFE-estimator-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TWFE estimator</a></span></li><li><span><a href="#Sun-and-Abraham" data-toc-modified-id="Sun-and-Abraham-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sun and Abraham</a></span></li><li><span><a href="#Chaisemartin-and-D'Haultfoeuille" data-toc-modified-id="Chaisemartin-and-D'Haultfoeuille-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Chaisemartin and D'Haultfoeuille</a></span></li></ul></div>

# 2008 ESA Analysis
By Zachary A. Goodman

Updated: 2021-05

This script performs the analysis in Goodman (2021).

## Init

In [ ]:
clear all

* set working directory
qui cd "../"

* set global directories
global raw_data ./data/raw_data
global temp_data ./data/temp_data
global gen_data ./data/gen_data
global figures ./tex/figures 
global tables ./tex/tables

* check dependencies
which eventstudyinteract
which reghdfe

* plot style
set scheme s1color

### Helper functions

In [ ]:
capture program drop get_data
program get_data
    use $gen_data/hh_year_week_panel, clear
    end

In [ ]:
capture program drop merge_dems
program merge_dems
    merge m:1 household_code using $gen_data/panelist_info.dta, keep(3) nogen
    sort household_code
    qui by household_code: ereplace wic_indicator_current = min(wic_indicator_current)
    qui by household_code: ereplace wic_indicator_ever_not_current = min(wic_indicator_ever_not_current)
    qui replace wic_indicator_current = 0 if mi(wic_indicator_current)
    qui replace wic_indicator_ever_not_current = 0 if mi(wic_indicator_ever_not_current)
    end

In [ ]:
* Sample-shrinking function for debugging

capture program drop keep_rand
program keep_rand
    args sample_size  // percent

    capture drop rnum
    bys household_code: gen rnum = runiform() if _n == 1
    by household_code: ereplace rnum = min(rnum)
    keep if rnum < `sample_size'
    end

In [ ]:
* Two-way Fixed Effect estimator

capture program drop twfe
program define twfe
    args outcome leads fname
    /*
    Two-way fixed effect estimator. 
    Must assume homogenous treatment effects and construct a 'never treated' group
    using the last treated cohort.
    
    Args
    outcome: y var
    leads: number of leads to include (lags limited by control group)
    fname: file name
    */

    capture restore
    preserve

    // keep_rand 0.3 // for shrinking sample for debugging
    
    di `leads'

    * drop obs after week of last treatment (2524)
    drop if week >= 2524
    //tab week_diff if week_treat == 2524  // verify last group has no treated obs

    * drop obs before `leads'
    drop if week_diff < -1 * `leads'

    * outcome in logs
    capture drop logy
    gen logy = asinh(`outcome')

    * factor-ize week_diff
    * leads (n = negative)
    capture drop g_*
    forvalues k = `leads'(-1)1 {
        qui gen g_n`k' = week_diff == -`k'
    }
    * lags
    forvalues k = 0/12 {
        qui gen g_`k' = week_diff == `k'
    }

    * drop reference category
    drop g_n2

    di "Running reghdfe..."
    
    reghdfe logy g_* [fw=projection_factor], ///
        absorb(household_code week) vce(cluster household_code) nocons

    //coefplot , vertical

    * get matrix of coef and se estimates
    matrix C = e(b)  // point estimates in relative time
    mata st_matrix("A", sqrt(st_matrix("e(V)")))  // variance estimates
    matrix A = vecdiag(A)
    matrix C = C \ A  // add row with standard errors
    matrix rownames C = coeff se
    matrix CT = C'  // transpose

    * export matrix
    mat2txt, matrix(CT) saving($temp_data/twfe/`fname'.txt) replace

    end

In [ ]:
* Sun and Abraham estimator

capture program drop sun_abraham
program define sun_abraham
    args outcome weeks fname
    /*
    outcome: y var
    weeks: number of leads and lags
    fname: file name
    */

    capture restore
    preserve

    di `weeks'

    // keep_rand  0.05  // shrink sample for debugging

    * gen logged outcome var
    capture drop logy
    gen logy = asinh(`outcome')

    * control cohort
    capture drop ctrl_cohort
    gen ctrl_cohort = week_treat == 2524

    * drop obs after week of last treatment (2524)
    drop if week >= 2524
    //tab week_diff if week_treat == 2524  // verify last group has no treated obs

    * generate leads and lags
    capture drop g_* 

    * leads (n = negative)
    forvalues k = `weeks'(-1)1 {
        qui gen g_n`k' = week_diff == -`k'
    }

    * lags
    forvalues k = 0/12 {
        qui gen g_`k' = week_diff == `k'
    }

    * drop reference weeks
    drop g_n`weeks' g_n2

    /*
    eventstudyinteract {y} {rel_time_list} [if] [in] [weight] ,
                   absorb(absvars) cohort(variable) control_cohort(variable)
                   [options covariates(varlist)]
    */
    di "Running eventstudyinteract..."

    eventstudyinteract logy g_* [fw=projection_factor], ///
        absorb(household_code week) cohort(week_treat) control_cohort(ctrl_cohort) ///
        vce(cluster household_code)


    * export coefficients

    matrix C = e(b_iw)  // point estimates in relative time
    mata st_matrix("A", sqrt(st_matrix("e(V_iw)")))  // variance estimates
    matrix C = C \ A  // add row with standard errors
    matrix rownames C = coeff se
    matrix CT = C'  // transpose
    //matrix list C
    //coefplot matrix(C[1]), se(C[2]) vertical
    mat2txt, matrix(CT) saving($temp_data/sun_abraham/`fname'.txt) replace
    
    end

## Descriptive stats

In [ ]:
get_data
merge_dems
gunique household_code
gunique household_code week

In [ ]:
tab week_diff

In [ ]:
sum

### Summary table

In [ ]:
* keep only necessary vars
keep household_code week_diff hh_method_paper-projection_factor ///
    items_deals-coupons_amount_saved calories-total_spent hh_* wic*

* keep only pretreatment obs
keep if week_diff < 0  // only pretreatment obs

* collapse to household level
collapse (mean) hh_method_paper-hh_child, by(household_code)

sum

In [ ]:
eststo clear
estpost tabstat hh_method_paper-hh_method_dd trips items_* final_price_paid-coupons_amount_saved ///
    calories-transfatgrams [fw=projection_factor], listwise ///
    statistics(mean sd) columns(statistics)

esttab, cells("mean(fmt(2)) sd") nomtitle nonumber

### How many imputed items?

In [ ]:
get_data

capture drop all*
gen all_items = imputed0 + imputed1 + imputed2 + imputed3 + imputed4 + imputed5
egen all_items_scanned = sum(all_items)
// egen all_items_scanned_2 = sum(items_food)
// sum all_items*

forvalues i = 0/5 {
    egen all_imputed_`i' = sum(imputed`i')
    di all_imputed_`i' / all_items_scanned
}

### Time series plots

In [ ]:
get_data
merge_dems

In [ ]:
capture program drop plot_ts
program plot_ts 
    args yvars byvar
    
    capture restore
    preserve

    capture drop week_diff
    gen week_diff = week - week_treat
    keep `yvars' `byvar' week_diff projection_factor
    collapse (mean) `yvars' [fw=projection_factor], by(week_diff `byvar')

    keep if inrange(week_diff, -12, 11)
    sum

    foreach y of varlist `yvars' {
        twoway (line `y' week_diff if `byvar' == 1) ///
            (line `y' week_diff if `byvar' == 0)
    }

    restore
    end

In [ ]:
local yvars = "total_spent"
local byvar = "hh_savings_two_months"
plot_ts "`yvars'" "`byvar'"

## TWFE estimator

In [ ]:
get_data

In [ ]:
* loop through outcomes and get coeffs

foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    twfe `v' 26 "all/`v'"
}

## Sun and Abraham

In [ ]:
get_data

In [ ]:
* loop through outcomes and get coeffs

foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    sun_abraham `v' 26 "all/`v'"
}

## Chaisemartin and D'Haultfoeuille

In [ ]:
* Try Chaisemartin & D'Haultfoeuille (2020)
* Very slow

capture restore
preserve

capture drop logx treat
gen logx = asinh(sugargrams)
gen treat = week_diff >= 0

* shrink sample to make sure program works
keep if week_diff >= -52
keep if week_diff <= 52
// gen randn = runiform()
// bys household_code: ereplace randn = max(randn)
// sort randn
// keep in 1/30000

* syntax: outcome group time_period_var treatment_var, options
did_multiplegt logx household_code week treat, switchers(in) breps(100) weight(projection_factor)

restore